In [168]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [169]:
df=pd.read_csv('KC_case_data .csv')

In [248]:
df['date']= pd.to_datetime(df['date'])

In [8]:
#MAU февраля:
df.query("'2020-02-01'<= date <= '2020-02-29'")\
  .device_id.nunique()

75032

In [15]:
##Количество установок в январе:
df.query("'2020-01-01'<= date <= '2020-01-31' and event == 'app_install'")\
  .event.count()
  

80297

In [ ]:
###Присвойте пользователям когорты по дню установки приложения и 
#посчитайте для них конверсию из установки в покупку в течение 7 дней. 
#Для какой когорты конверсия была наибольшей?
#считаем пользователя сконвертировавшимся, если с момента установки 
#до совершения первой покупки прошло не более 7 дней.

In [287]:
#отбираем пользователей с установкой
df_install = df.query("event == 'app_install'")\
               .rename(columns={'date':'date_install'})
df_install

,date_install,event,purchase_sum,os_name,device_id,gender,city,utm_source
2515061,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads
2515062,2020-01-01,app_install,NaN,ios,1311583,male,Moscow,-
2515063,2020-01-01,app_install,NaN,android,23006325,male,Saint-Petersburg,-
2515064,2020-01-01,app_install,NaN,android,5645652,male,Moscow,-
2515065,2020-01-01,app_install,NaN,ios,8798567,male,Saint-Petersburg,referal
...,...,...,...,...,...,...,...,...
2669653,2020-03-31,app_install,NaN,ios,19717968,female,Moscow,-
2669654,2020-03-31,app_install,NaN,android,32966665,female,Saint-Petersburg,-
2669655,2020-03-31,app_install,NaN,ios,6335964,female,Moscow,referal
2669656,2020-03-31,app_install,NaN,android,29155826,male,Saint-Petersburg,vk_ads


In [288]:
#отбираем пользователей с минимальной датой покупки
df_purchase = df.query("event == 'purchase'")\
                .rename(columns={'date':'date_purchase'})\
                .groupby('device_id',as_index=False)\
                .agg({'date_purchase':'min'})
df_purchase

,device_id,date_purchase
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4120,2020-01-07
4,4125,2020-01-07
...,...,...
70870,35321120,2020-03-25
70871,35326839,2020-03-28
70872,35342310,2020-03-27
70873,35367797,2020-03-29


In [289]:
#объединяем датафреймы с  установками и покупками
df_cr = df_install.merge(df_purchase,how='left',on='device_id')
df_cr

,date_install,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_purchase
0,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,2020-01-10
1,2020-01-01,app_install,NaN,ios,1311583,male,Moscow,-,NaT
2,2020-01-01,app_install,NaN,android,23006325,male,Saint-Petersburg,-,2020-01-01
3,2020-01-01,app_install,NaN,android,5645652,male,Moscow,-,NaT
4,2020-01-01,app_install,NaN,ios,8798567,male,Saint-Petersburg,referal,NaT
...,...,...,...,...,...,...,...,...,...
154592,2020-03-31,app_install,NaN,ios,19717968,female,Moscow,-,NaT
154593,2020-03-31,app_install,NaN,android,32966665,female,Saint-Petersburg,-,NaT
154594,2020-03-31,app_install,NaN,ios,6335964,female,Moscow,referal,NaT
154595,2020-03-31,app_install,NaN,android,29155826,male,Saint-Petersburg,vk_ads,NaT


In [290]:
#считаем разницу дней от установки до первой покупки
df_cr['days'] = df_cr.date_purchase-df_cr.date_install

In [291]:
#отмечаем пользователей, у которых от момента установки до покупки прошло не более 7 дней
df_cr['conv'] = df_cr['days']<= '7 days'
df_cr

,date_install,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_purchase,days,conv
0,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,2020-01-10,9 days,False
1,2020-01-01,app_install,NaN,ios,1311583,male,Moscow,-,NaT,NaT,False
2,2020-01-01,app_install,NaN,android,23006325,male,Saint-Petersburg,-,2020-01-01,0 days,True
3,2020-01-01,app_install,NaN,android,5645652,male,Moscow,-,NaT,NaT,False
4,2020-01-01,app_install,NaN,ios,8798567,male,Saint-Petersburg,referal,NaT,NaT,False
...,...,...,...,...,...,...,...,...,...,...,...
154592,2020-03-31,app_install,NaN,ios,19717968,female,Moscow,-,NaT,NaT,False
154593,2020-03-31,app_install,NaN,android,32966665,female,Saint-Petersburg,-,NaT,NaT,False
154594,2020-03-31,app_install,NaN,ios,6335964,female,Moscow,referal,NaT,NaT,False
154595,2020-03-31,app_install,NaN,android,29155826,male,Saint-Petersburg,vk_ads,NaT,NaT,False


In [292]:
#считаем количество пользователей с установками и с покупками
df_cr_tot = df_cr.groupby(['date_install'],as_index=False)\
                 .agg({'conv':lambda x: x.sum(),
                       'device_id':lambda x: x.nunique()})
df_cr_tot

,date_install,conv,device_id
0,2020-01-01,1408,3579
1,2020-01-02,1186,3144
2,2020-01-03,834,2402
3,2020-01-04,639,1831
4,2020-01-05,587,1671
...,...,...,...
86,2020-03-27,277,1199
87,2020-03-28,209,1091
88,2020-03-29,223,1117
89,2020-03-30,204,994


In [293]:
#считаем конверсию из установки в покупку
df_cr_tot['cr'] = df_cr_tot['conv']/df_cr_tot['device_id']*100

In [294]:
#находим дату с самым высоким cr
df_cr_tot = df_cr_tot.sort_values('cr',ascending=False)
df_cr_tot
#самый высокий показатель cr = 39.3% и был у когорты 01-01-2021 

,date_install,conv,device_id,cr
0,2020-01-01,1408,3579,39.340598
8,2020-01-09,558,1424,39.185393
14,2020-01-15,1650,4310,38.283063
13,2020-01-14,1973,5173,38.140344
1,2020-01-02,1186,3144,37.722646
...,...,...,...,...
77,2020-03-18,235,1171,20.068318
88,2020-03-29,223,1117,19.964190
81,2020-03-22,251,1261,19.904837
87,2020-03-28,209,1091,19.156737


In [107]:
####С какого платного маркетингового канала пришло больше всего новых пользователей? 
df.groupby('utm_source',as_index=False)\
  .agg({'device_id':'nunique'})\
  .sort_values('utm_source',ascending=False)

,utm_source,device_id
6,yandex-direct,40712
5,vk_ads,34086
4,referal,15926
3,instagram_ads,31048
2,google_ads,38096
1,facebook_ads,25959
0,-,52273


In [262]:
#####Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов.
df_voronka = df.groupby('event',as_index=False)\
               .agg({'device_id':'nunique'})
df_voronka
#исходя из полученных данных больше всего пользователй ушло на этапе перехода в корзину.

,event,device_id
0,app_install,154597
1,app_start,190884
2,choose_item,155691
3,purchase,70875
4,register,78310
5,search,184488
6,tap_basket,125414


In [ ]:
######Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?

In [263]:
#отбираем пользователей, которые открыли приложение и считаем их количество
df_app_start = df.query("event == 'app_start'")\
                .groupby('utm_source', as_index=False)\
                .agg({'device_id':'nunique'})\
                .rename(columns={'device_id':'count_app_start'})
df_app_start

,utm_source,count_app_start
0,-,52268
1,facebook_ads,25957
2,google_ads,38092
3,instagram_ads,31045
4,referal,15924
5,vk_ads,34079
6,yandex-direct,40707


In [266]:
#отбираем пользователей с первой покупкой и считаем их количество
df_purchase = df.query("event == 'purchase'")\
                .groupby(['device_id','utm_source'],as_index=False)\
                .agg({'date':'min'})\
                .rename(columns={'date':'first_purchase'})\
                .groupby('utm_source',as_index=False)\
                .agg({'device_id':'nunique'})\
                .rename(columns={'device_id':'count_purchase'})
df_purchase

,utm_source,count_purchase
0,-,16598
1,facebook_ads,9017
2,google_ads,11339
3,instagram_ads,10762
4,referal,6362
5,vk_ads,12364
6,yandex-direct,12028


In [267]:
#объединяем датафреймы с открытием приложения и с первыми покупками 
df_cr = df_app_start.merge(df_purchase,on= 'utm_source')
df_cr

,utm_source,count_app_start,count_purchase
0,-,52268,16598
1,facebook_ads,25957,9017
2,google_ads,38092,11339
3,instagram_ads,31045,10762
4,referal,15924,6362
5,vk_ads,34079,12364
6,yandex-direct,40707,12028


In [268]:
#считаем конверсию в покупку
df_cr['cr'] = df_cr.count_purchase/df_cr.count_app_start*100
df_cr
#самая низкая конверсия у яндекса

,utm_source,count_app_start,count_purchase,cr
0,-,52268,16598,31.755567
1,facebook_ads,25957,9017,34.738221
2,google_ads,38092,11339,29.767405
3,instagram_ads,31045,10762,34.665808
4,referal,15924,6362,39.952273
5,vk_ads,34079,12364,36.280407
6,yandex-direct,40707,12028,29.547744


In [ ]:
#######Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? 
#(учитываются только первые покупки пользователей)

In [274]:
#отбираем дату первой покупки
df_purchase = df.query("event == 'purchase'")\
                .groupby(['utm_source','purchase_sum'],as_index=False)\
                .agg({'date':'min'})
df_purchase

,utm_source,purchase_sum,date
0,-,61.0,2020-01-11
1,-,61.5,2020-01-03
2,-,62.0,2020-01-02
3,-,62.5,2020-01-03
4,-,63.0,2020-01-05
...,...,...,...
19151,yandex-direct,7861.5,2020-01-26
19152,yandex-direct,7865.0,2020-03-21
19153,yandex-direct,7872.5,2020-01-07
19154,yandex-direct,7876.5,2020-02-11


In [276]:
#считаем медианный первый чек
df_median_purchase = df_purchase.groupby('utm_source',as_index=False)\
                                .agg({'purchase_sum':'median'})\
                                .sort_values('purchase_sum',ascending=False)
df_median_purchase
#самы высокий медианный первый чек у пользователей, пришедшиз из VK

,utm_source,purchase_sum
0,-,2800.25
5,vk_ads,2348.00
3,instagram_ads,2073.00
6,yandex-direct,1927.00
2,google_ads,1630.00
1,facebook_ads,1544.50
4,referal,600.00


In [ ]:
########Данные по затратам на рекламу:

#Яндекс – 10 491 707 руб.

#Гугл – 10 534 878 руб.

#Фейсбук – 8 590 498 руб.

#Инстаграм – 8 561626 руб.

#ВК – 9 553 531руб.

#Расходы на реферальную программу: если пользователь приведет друга и последний совершит первую покупку, 
#то оба получат по 100 рублей.

#Какой платный канал привлечения имеет самый высокий ROMI? 

In [279]:
#считаем прибыль
profit = df.query("event == 'purchase'")\
                .purchase_sum.sum()
profit  

100256960.5

In [282]:
#считаем расходы на реферальную программу
referal = df.query("event == 'purchase'")\
            .query("utm_source == 'referal'")\
            .device_id.nunique()*200
referal

1272400

In [192]:
yandex = 10491707
google = 10534878
facebook = 8590498
instagram = 8561626
vk = 9553531

In [283]:
ROMI_yandex = (profit-yandex)/yandex
ROMI_yandex

8.555829237320486

In [284]:
ROMI_google = (profit-google)/google
ROMI_google

8.516670292717201

In [285]:
ROMI_facebook = (profit-facebook)/facebook

ROMI_facebook

10.670680849934428

In [236]:
ROMI_instagram = (profit-instagram)/instagram
ROMI_instagram

10.710037380749872

In [237]:
ROMI_vk = (profit-vk)/vk
ROMI_vk

9.494230928857613

In [238]:
ROMI_referal = (profit-referal)/referal
ROMI_referal

77.79358731530965

In [ ]:
#самый высокий показатель ROMI у реферальной программы